In [1]:
from Attention import MultiHeadCrossAttention

In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

In [12]:
# Example usage
embed_dim = 768
num_heads = 8
query = torch.rand(32, 20, embed_dim)  # Batch size: 32, Sequence length: 10
key = torch.rand(32, 10, embed_dim)    # Sequence length: 20
value = torch.rand(32, 10, embed_dim)  # Sequence length: 20

multihead_cross_attention = MultiHeadCrossAttention(embed_dim, num_heads)
output = multihead_cross_attention(query, key, value)
print(output.shape)  # Expected output shape: (32, 10, 64)


torch.Size([32, 20, 768])


In [9]:
# Example usage
embed_dim = 64
num_heads = 8
query = torch.rand(32, 10, embed_dim)  # Batch size: 32, Sequence length: 10
key = torch.rand(32, 20, embed_dim)    # Sequence length: 20
value = torch.rand(32, 20, embed_dim)  # Sequence length: 20

multihead_cross_attention = MultiHeadCrossAttention(embed_dim, num_heads)
output = multihead_cross_attention(query, key, value)
print(output.shape)  # Expected output shape: (32, 10, 64)


torch.Size([32, 10, 64])


In [28]:
from transformers import BertTokenizer, BertModel
import torch

# 初始化 BERT tokenizer 和模型
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# 输入单词
input_text = "hello"
inputs = tokenizer(input_text, return_tensors='pt')
# 获取模型的输出
outputs = model(**inputs)

# 获取最后一层的隐藏状态
last_hidden_states = outputs.last_hidden_state

print(last_hidden_states)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tensor([[[-0.3061,  0.2622, -0.1896,  ..., -0.1651,  0.1014,  0.4119],
         [-0.7390, -0.0336,  0.3932,  ..., -0.1818, -0.1839, -0.2185],
         [ 0.5801,  0.0627, -0.2637,  ...,  0.3963, -0.5684, -0.4924]]],
       grad_fn=<NativeLayerNormBackward0>)


In [45]:
input_text = "hello, my name is chris. This is my dog zoe"
inputs = tokenizer(input_text, return_tensors='pt')

In [69]:
inputs_ids = inputs['input_ids']
inputs_mask = inputs['attention_mask']
print(inputs_ids.shape)
inputs_ids = inputs_ids.unsqueeze(1)
print(inputs_ids.shape)
bs,rnt,sl = inputs_ids.shape
inputs_ids = inputs_ids.view(-1,sl)
print(inputs_ids.shape)


torch.Size([1, 14])
torch.Size([1, 1, 14])
torch.Size([1, 14])


In [72]:
outputs = model(inputs_ids,inputs_mask)[0][:,0,:]
print(outputs.shape)
outputs = outputs.view(bs,rnt,-1)
print(outputs.shape)
outputs = outputs.permute(2,0,1)
print(outputs.shape)

torch.Size([1, 768])
torch.Size([1, 1, 768])
torch.Size([768, 1, 1])


In [82]:
word = "hello"
word = tokenizer(word, return_tensors='pt')
word_ids = word['input_ids']
word_mask = word['attention_mask']
out_word = model(word_ids)[0][:,1,:]
out_word.shape

torch.Size([1, 768])

In [47]:
inputs_ids = inputs_ids.view(-1,seq_len)

In [48]:
inputs_ids.shape

torch.Size([1, 14])

In [61]:
import torch

# 创建一个 2D 张量
tensor = torch.tensor([[1, 2, 3], [4, 5, 6]])

# 查看原始张量的形状
print("Original shape:", tensor.shape)  # 输出: torch.Size([2, 3])

# 使用 view 将其转换为 1D 张量
flattened_tensor = tensor.view(-1,6)

# 查看转换后的张量形状
print("Flattened shape:", flattened_tensor.shape)  # 输出: torch.Size([6])


Original shape: torch.Size([2, 3])
Flattened shape: torch.Size([1, 6])


In [79]:
import torch

# 创建一个1D张量
tensor = torch.tensor([1, 2, 3])
print(tensor)
# 查看原始张量的形状
print("Original shape:", tensor.shape)  # 输出: torch.Size([3])

# 使用expand将其扩展为2D张量
expanded_tensor = tensor.expand(5, 3)

# 查看扩展后的张量形状
print("Expanded shape:", expanded_tensor.shape)  # 输出: torch.Size([2, 3])
print(expanded_tensor)


tensor([1, 2, 3])
Original shape: torch.Size([3])
Expanded shape: torch.Size([5, 3])
tensor([[1, 2, 3],
        [1, 2, 3],
        [1, 2, 3],
        [1, 2, 3],
        [1, 2, 3]])


In [109]:
def dot_Attention(Q,K,V):
    attention_weights = torch.matmul(Q,K.transpose(2,1))
    attention_weights = F.softmax(attention_weights,-1)
    output = torch.matmul(attention_weights, V)
    return output

In [149]:
ctx = "Zoe is my pet dog. She is very cute. She is a golden retriever. She is 2 years old. She is very friendly."
gloss = "Zoe is a golden retriever. She is 2 years old. She is very friendly."

ctx = tokenizer(ctx, return_tensors='pt')
gloss = tokenizer(gloss, return_tensors='pt')

ctx_ids,ctx_mask,gloss_ids,gloss_mask = ctx['input_ids'],ctx['attention_mask'],gloss['input_ids'],gloss['attention_mask']

ctx_out = model(ctx_ids,ctx_mask)[0]
print("context输出的维度为:",ctx_out.shape)
gloss_out = model(gloss_ids,gloss_mask)[0][:,0,:]
print("gloss输出的维度为:",gloss_out.shape)

poly_code_ids = torch.arange(10,dtype=torch.long)
embedding_layer = nn.Embedding(10,768)
poly_code = embedding_layer(poly_code_ids)

ctx_emd = dot_Attention(poly_code,ctx_out,ctx_out)
print("完成poly之后的context维度为::",ctx_emd.shape)

gloss_out = gloss_out.view(1,1,-1)
print("gloss的维度为:",gloss_out.shape)
gloss_emd = gloss_out.permute(1,0,2)
print("gloss的维度为:",gloss_emd.shape)
gloss_emd = gloss_emd.expand(1,1,768)
print("扩充完的gloss的维度是:",gloss_emd.shape)
ctx_emb = dot_Attention(gloss_emd,ctx_emd,ctx_emd)
print("执行完注意力机制之后的ctx的维度是:",ctx_emb.shape)
gloss_emd = gloss_emd.permute(1,0,2)
print("gloss转置后的维度是:",gloss_emd.shape)

context输出的维度为: torch.Size([1, 31, 768])
gloss输出的维度为: torch.Size([1, 768])
完成poly之后的context维度为:: torch.Size([1, 10, 768])
gloss的维度为: torch.Size([1, 1, 768])
gloss的维度为: torch.Size([1, 1, 768])
扩充完的gloss的维度是: torch.Size([1, 1, 768])
执行完注意力机制之后的ctx的维度是: torch.Size([1, 1, 768])


In [121]:
poly_ctx = torch.rand(20,10,768)
gloss_embedding = torch.rand(20,20,768)

ctx_emb = dot_Attention(gloss_embedding,poly_ctx,poly_ctx)
ctx_emb = ctx_emb.permute(2,0,1)
gloss_embedding = gloss_embedding.permute(2,0,1)
result = ctx_emb*gloss_embedding
result.shape

torch.Size([768, 20, 20])

In [139]:
A = torch.rand(10,768)
B = torch.rand(768,10)
C = torch.matmul(A,B)
C.shape

torch.Size([10, 10])

In [141]:
import torch

# 创建两个示例张量
poly_n = 10
bs,dim = 40, 768
A = torch.randn(poly_n, bs, dim)
B = torch.randn(poly_n, dim, bs)

# 使用 torch.matmul 进行矩阵乘法
C = torch.matmul(A, B)

# 查看结果形状
print("A shape:", A.shape)  # 输出: torch.Size([2, 3, 4])
print("B shape:", B.shape)  # 输出: torch.Size([2, 4, 5])
print("C shape:", C.shape)  # 输出: torch.Size([2, 3, 5])


A shape: torch.Size([10, 40, 768])
B shape: torch.Size([10, 768, 40])
C shape: torch.Size([10, 40, 40])


In [148]:
p = torch.arange(10,dtype=torch.long)
print(p.unsqueeze(0).shape)
embedding_layer = nn.Embedding(10,768)
poly_code = embedding_layer(poly_code_ids)
print(poly_code.shape)

torch.Size([1, 10])
torch.Size([10, 768])


In [164]:
# 创建两个示例张量
poly_n = 10
bs,dim = 40, 768
ctx = torch.randn(bs, poly_n, dim)
gloss = torch.randn(bs,dim)
print("这一步的维度分别是为Gloss:",gloss.shape,"\n和context:",ctx.shape)

gloss = gloss.expand(poly_n,bs,dim)
gloss = gloss.permute(1,0,2)
print("扩展后的gloss维度为:",gloss.shape)
ctx_emb = dot_Attention(gloss,ctx,ctx)
print("交叉注意力点积之后的context为:",ctx_emb.shape)
ctx_emb = ctx_emb.permute(1,0,2)
gloss = gloss.permute(1,2,0)
print("这一步的维度分别是为Gloss:",gloss.shape,"\n和context:",ctx.shape)
representation = torch.matmul(ctx_emb,gloss)
print("最终的特征融合结果为:",representation.shape)

这一步的维度分别是为Gloss: torch.Size([40, 768]) 
和context: torch.Size([40, 10, 768])
扩展后的gloss维度为: torch.Size([40, 10, 768])
交叉注意力点积之后的context为: torch.Size([40, 10, 768])
这一步的维度分别是为Gloss: torch.Size([10, 768, 40]) 
和context: torch.Size([40, 10, 768])
最终的特征融合结果为: torch.Size([10, 40, 40])


In [215]:
# 你的测试句子列表
sentences = [
    "Hello, how are you?",
    "Transformers are very powerful.",
    "Do you like machine learning?"
]

glosses= [
    "Hello, how are you?",
    "Transformers are very powerful.",
    "Do you like machine learning?"
]

words = [
    "hello",
    "joe",
    "bye"
]


# 使用分词器处理文本数据

ctx_inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')
gloss_inputs = tokenizer(glosses, padding=True, truncation=True, return_tensors='pt')
words_inputs = tokenizer(words, padding=True, truncation=True, return_tensors='pt')

# 推理，获取隐藏状态
with torch.no_grad():
    ctx_output = model(**ctx_inputs)[0]
    gloss_output = model(**gloss_inputs)[0][:,0,:]
    words_output = model(words_inputs["input_ids"])[0][:,1,:]

print("ctx_output的维度为:",ctx_output.shape)
print("gloss_output的维度为:",gloss_output.shape)
print("wprd_output的维度为:",words_output.shape)
print("++++++++++++++++++++++++++++++++++++++\n")

poly_code_ids = torch.arange(10,dtype=torch.long)
embedding_layer = nn.Embedding(10,768)
poly_code = embedding_layer(poly_code_ids)
print("poly_code的维度为:",poly_code.shape)
ctx_emb = dot_Attention(poly_code,ctx_output,ctx_output)
print("ctx_emb的维度为:",ctx_emb.shape)

gloss_emb = gloss_output.expand(10,3,768).transpose(0, 1)
#gloss_emb = gloss_emb.permute(1,0,2)
print("gloss_emb的维度为:",gloss_emb.shape)

words_emb = words_output.expand(10,3,768).transpose(0, 1)
print("words_emb的维度为:",words_emb.shape)
print("++++++++++++++++++++++++++++++++++++++\n")

gloss_emb = multihead_cross_attention(words_emb,gloss_emb,gloss_emb)
context_emb = multihead_cross_attention(words_emb,ctx_emb,ctx_emb)


ctx_emb = dot_Attention(gloss_emb,ctx_emb,ctx_emb)
print("ctx_emb的维度为:",ctx_emb.shape)

ctx_emb = ctx_emb.permute(1,0,2)
gloss_emb = gloss_emb.permute(1,2,0)
representation = torch.matmul(ctx_emb,gloss_emb)
print("representation的维度为:",representation.shape)
representation = representation.sum(0)
print("representation的维度为:",representation.shape)

ctx_output的维度为: torch.Size([3, 8, 768])
gloss_output的维度为: torch.Size([3, 768])
wprd_output的维度为: torch.Size([3, 768])
++++++++++++++++++++++++++++++++++++++

poly_code的维度为: torch.Size([10, 768])
ctx_emb的维度为: torch.Size([3, 10, 768])
gloss_emb的维度为: torch.Size([3, 10, 768])
words_emb的维度为: torch.Size([3, 10, 768])
++++++++++++++++++++++++++++++++++++++

ctx_emb的维度为: torch.Size([3, 10, 768])
representation的维度为: torch.Size([10, 3, 3])
representation的维度为: torch.Size([3, 3])


In [194]:
gloss_ids = gloss_inputs["input_ids"]
gloss_ids = gloss_ids.unsqueeze(1)
print("gloss_ids的维度为:",gloss_ids.shape)
bs,rnt,sl = gloss_ids.shape
print("bs,rnt,sl的维度为:",bs,rnt,sl)
gloss_ids = gloss_ids.squeeze(1)
print("gloss_ids的维度为:",gloss_ids.shape)

gloss_ids的维度为: torch.Size([3, 1, 8])
bs,rnt,sl的维度为: 3 1 8
gloss_ids的维度为: torch.Size([3, 8])


In [201]:
poly_codes_ids = torch.arange(10,dtype=torch.long)
embedding_layer = nn.Embedding(10,768)
print("poly_codes_ids的维度为:",poly_codes_ids.shape)
poly_codes = embedding_layer(poly_codes_ids)
print("poly_codes的维度为:",poly_codes.shape)

poly_codes_ids = poly_codes_ids.expand(bs,10)
print("poly_codes_ids的维度为:",poly_codes_ids.shape)

poly_codes_ids的维度为: torch.Size([10])
poly_codes的维度为: torch.Size([10, 768])
poly_codes_ids的维度为: torch.Size([3, 10])
